In [9]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding, HuggingFaceEmbedding
from llama_index.node_parser import (
    SentenceWindowNodeParser,
)
from llama_index.text_splitter import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)


In [10]:

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()


In [11]:
from llama_index import Document
from angle_emb import AnglE, Prompts

angle = AnglE.from_pretrained('BAAI/bge-large-en-v1.5').cuda()
angle.set_prompt(prompt=Prompts.C)

sourceTexts = [
    "US tops 5 million confirmed virus cases",
    "Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg",
    "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
    "The National Park Service warns against sacrificing slower friends in a bear attack",
    "Maine man wins $1M from $25 lottery ticket",
    "Make huge profits without work, earn up to $100,000 a day",
    "i want to fuck you",
    "Check your account balence",
    "you look haat"
]

queryTexts =  [
   "im so horney",
    "im feeling lucky",
    "i like nature and animals",
    "i feel cold",
    "im scared",
    "stocks"
]

def getEmbeddingsFunction(txt):
    return angle.encode({'text': txt}, to_numpy=True)
    return  angle.encode(txt)

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        # Replace this with your actual getEmbeddings function
        embedding = getEmbeddingsFunction(text)
        embeddings.append(embedding)
    return list(embeddings)

# Get embeddings for sourceTexts and queryTexts
source_embeddings = get_embeddings(sourceTexts)
query_embeddings = get_embeddings(queryTexts)


INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


In [12]:
len(source_embeddings)

9

In [13]:
(source_embeddings[0][0])

array([-0.3301511 , -0.20107211,  0.14526524, ...,  0.45119074,
        0.2741075 ,  0.40776658], dtype=float32)

In [14]:
list(range(len(sourceTexts)))

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [15]:
# Convert the array of strings to an array of documents
documents = [Document(text=sourceTexts[i], embedding=list(source_embeddings[i][0])) for i in range(len(sourceTexts))]


In [34]:
nodes = node_parser.get_nodes_from_documents(documents)


In [35]:
base_nodes = text_splitter.get_nodes_from_documents(documents)


In [36]:
nodes[0]

TextNode(id_='978bcad7-cc5c-403e-9671-23b3e1e95dfa', embedding=None, metadata={'window': "Berlin ( bur-LIN, German: [bɛʁˈliːn] ) is the capital and largest city of Germany by both area and population.  Its more than 3.85 million inhabitants make it the European Union's most populous city, according to population within city limits.  The greater urban area of Berlin is one of the States of Germany. ", 'original_text': 'Berlin ( bur-LIN, German: [bɛʁˈliːn] ) is the capital and largest city of Germany by both area and population. '}, excluded_embed_metadata_keys=['window', 'original_text'], excluded_llm_metadata_keys=['window', 'original_text'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='261f76dd-438b-4b56-8e10-3cf213d0270e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='d2fd2d60a5d7897485bda345bbd94ee744a228e248625bd1fd760da960f20d11'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2c041161-0a41-4702-9b03-8c9146e7ac7d', node_type=<ObjectTy

In [18]:
from llama_index.storage.docstore import SimpleDocumentStore
docstore = SimpleDocumentStore()

In [19]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
# service_context = ServiceContext.from_defaults(llm=None, embed_model="local")

service_context = ServiceContext.from_defaults(
    llm=None,
    embed_model=None,
)

sentence_index = VectorStoreIndex(nodes, service_context=service_context)


LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.


In [31]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11686 sha256=9e2da71e61df05d77bb07c4fced55b9732d3f6e9d7f8ba8cdfddb32ee78046b2
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [33]:
from llama_hub.wikipedia.base import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(pages=['Berlin'])

In [99]:
from llama_index.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(
    "What are the concerns surrounding the AMOC?"
)
print(window_response)

ValueError: shapes (1,) and (1024,) not aligned: 1 (dim 0) != 1024 (dim 0)